# Signed Distance Functions

This notebook demonstrates the basic principles of signed distance functions (SDFs).
All examples are in plain Python and in 2D only.
For more practical SDF implementations in Python, check out Michael Fogleman's [sdf](https://github.com/fogleman/sdf) module or [Yann Büchaus's fork](https://github.com/nobodyinperson/sdf)

In [ ]:
from IPython.display import display
from PIL import Image as PImage
import math

In [ ]:
i = PImage.new("L", (300,300))
display(i)

In [ ]:
# Helper function:
def length(p:tuple):
    """Returns the length of a 2D vector p (same as the distance of the point p from the origin)"""
    x, y = p
    return math.sqrt(x*x + y*y)

In [ ]:
# My first little SDF

def circle(p: tuple[int|float], r: float):
    """Returns the distance of a point p to the edge of a circle around (0,0) with radius r.
       For points on the edge of the circle return 0, for points inside the circle a negative value,
       for points outside a positive value"""
    return length(p) - r

In [ ]:
assert circle(p=(0,3), r=2) == 1.0
assert circle(p=(2,0), r=2) == 0.0
assert circle(p=(1,0), r=2) == -1.0

Ok, now that this function works, let's draw a circle with it.
We do this by testing for every pixel in the image whether it is inside the circle, outside, or on its edge.

In [ ]:
%%time
for x in range(i.width):
    for y in range(i.height):
        value = circle((x,y), 80)
        if value == 0.0: # edge
            color = 255            
        elif value < 0.0: # inside
            color = 128
        elif value > 0.0: # outside
            color = 0
        i.putpixel((x, y), color)
display(i)

Ok, seems to work. But there are two problems:

1. the white edge is only shown as a few individual pixels
2. the circle is centered around the origin

Let's fix both problems...

**Drawing a better border**:
The reason why the border is only shown as individual pixels is that we only draw a border pixel if it lies exactly on the mathematically defined edge of the circle, i.e. if `math.sqrt(x*x+y*y) == r`
As we only have pixels at integer locations, there are only a few of them exactly on the circle's edge. 
In our example, there are just four points:

In [ ]:
for x in range(300):
    for y in range(300):
        if math.sqrt(x*x+y*y) == 80:
            print(x,y)

So, let's accept a pixel as being on the edge of the circle if it is within half a pixel of the mathematically-defined edge.

In [ ]:
%%time
for x in range(i.width):
    for y in range(i.height):
        value = circle((x,y), 80)
        if abs(value) <= 0.5: # edge
            color = 255
        elif value < 0.0: # inside
            color = 128
        elif value > 0.0: # outside
            color = 0
        i.putpixel((x, y), color)
display(i)

Great. We can increase the border size by changing the '0.5' value to larger values. 
Also, we could easily add antialiasing/smoothing of the border by checking how close to the edge a border pixel is and then adjusting its brightness accordingly.
(Try it yourself!)

In [ ]:
%%time
border = 6.0
for x in range(i.width):
    for y in range(i.height):
        value = circle((x,y), 80)
        if abs(value) <= border / 2: # edge
            color = int(255 * (1 - abs(value) / (border / 2)))
        elif value < 0.0: # inside
            color = 0  # why did I change the inside to black? How would the antialiasing equation have to be changed in order to work with arbitrary background colors?
        elif value > 0.0: # outside
            color = 0
        i.putpixel((x, y), color)
display(i)

**Arbitrary placement of the SDF shape:**
The trick for placing an SDF-defined shape anywhere on the screen is to transform the position of the point to be checked first.
In order to check whether a point p(x,y) is within a circle at position (cx,cy) and radius r on the screen, we first subtract (cx,cy) from the point and then check whether the resulting point p'(x-cx,y-cy) lies within the circle of radius r around the origin (0,0).

In [ ]:
def circle(p: tuple[int|float], r: float, c: tuple[int|float] = (0,0)):
    """Returns the distance of a point p to the edge of a circle around point c with radius r.
       For points on the edge of the circle return 0, for points inside the circle a negative value,
       for points outside a positive value"""
    x, y = p  # extract coordinates
    cx, cy = c 
    p = (x - cx, y - cy)
    return length(p) - r

In [ ]:
%%time
border = 1.0

for x in range(i.width):
    for y in range(i.height):
        value = circle((x,y), 80, (150,150))
        if abs(value) <= border/2:
            color = 255
        elif value < 0.0:
            color = 128
        elif value > 0.0:
            color = 0
        i.putpixel((x, y), color)
display(i)

# Other shapes
We can also draw other shapes using SDFs. 
Inigo Quilez has a [nice collection](https://iquilezles.org/articles/distfunctions2d/) of 2D (and 3D) SDFs. 
As his SDFs are written in GLSL, they need to be translated into Python.

In [ ]:
def rect(p: tuple[int|float], size: tuple[int|float], center: tuple[int|float] = (0,0)):
    # unpack tuples because Python does not allow e.g., subtracting one tuple from another (numpy does)
    x, y = p
    cx, cy = center
    w, h = size
    # transform point
    x, y = (x - cx, y - cy)
    # calculate rect SDF (https://iquilezles.org/articles/distfunctions2d/)
    dx, dy = (abs(x) - w, abs(y) - h);
    return length((max(dx, 0.0), max(dy, 0.0))) + min(max(dx,dy),0.0)

In [ ]:
%%time
border = 2.0

for x in range(i.width):
    for y in range(i.height):
        value = rect((x,y), (80,30), (150,150))
        if abs(value) <= border/2:
            color = 255
        elif value < 0.0:
            color = 128
        elif value > 0.0:
            color = 0
        i.putpixel((x, y), color)
display(i)

# Boolean Operations
A nice thing about SDFs is that you can combine them using Boolean operations.
(An explanation is outside the scope of this quick tutorial):

In [ ]:
%%time
# Union: min(sdf1, sdf2)
border = 2.0

for x in range(i.width):
    for y in range(i.height):
        value = min(circle((x,y), 50, (150,150)),
                    rect((x,y), (100,20), (150,150)))
        if abs(value) <= border/2:
            color = 255
        elif value < 0.0:
            color = 128
        elif value > 0.0:
            color = 0
        i.putpixel((x, y), color)
display(i)

In [ ]:
%%time
# Intersection: max(sdf1, sdf2)
border = 2.0

for x in range(i.width):
    for y in range(i.height):
        value = max(circle((x,y), 50, (150,150)),
                rect((x,y), (100,20), (150,150)))
        if abs(value) <= border/2:
            color = 255
        elif value < 0.0:
            color = 128
        elif value > 0.0:
            color = 0
        i.putpixel((x, y), color)
display(i)

In [ ]:
%%time
# Difference: max(sdf1, -sdf2)
border = 2.0

for x in range(i.width):
    for y in range(i.height):
        value = max(circle((x,y), 50, (150,150)),
                -rect((x,y), (100,20), (150,150)))
        if abs(value) <= border/2:
            color = 255
        elif value < 0.0:
            color = 128
        elif value > 0.0:
            color = 0
        i.putpixel((x, y), color)
display(i)

In [ ]:
# To do (if you want): draw a smilie face or a car or something else using SDFs.

In [ ]:
# To do (if you want): create a set of functions that can be easily combined, e.g.:
my_sdf = union(circle(80, (150,150)),
               rect((80,30),(150,230)))
my_canvas.draw(my_sdf)